# Part 2 - Where to enrich? (what are study areas?)

## Enriching Study Areas
GeoEnrichment uses the concept of a study area to define the location of the point or area that you want to enrich with additional information or create reports about. The accepted forms of study areas are:
1. Street address locations
    - a. Single line input
    - b. Multiple field input
2. Point, line and polygon geometries 
3. Buffered study areas
4. Named statistical areas

Before we look at the exmaples of study areas, let's understand the concept of Data collections and analysis variables. We will look at Data collections in detail in a later section.

### Data collections and analysis variables
GeoEnrichment uses the concept of a data collection to define the data attributes (analysis variables) returned by the enrichment service. A data collection is a preassembled list of attributes that will be used to enrich the input features. Collection attributes can describe various types of information, such as demographic characteristics and geographic context of the locations or areas submitted as input features. We will introduce the concept of data collections here and look at the details in the next guide.

The `Country` class can be used to discover the data collections, sub-geographies and available reports for a country. When working with a particular country, you will find it convenient to get a reference to it using the Country.get() method. 

The `data_collections` property of a `Country` object lists a combination of available data collections and analysis variables for each data collection as a [Pandas](https://pandas.pydata.org/) dataframe. 

Once we know the data collection we would like to use, we can look at all the unique `analysisVariable`  available in that data collection.

In [1]:
# Import Libraries
from arcgis.gis import GIS
from arcgis.geoenrichment import Country, enrich, BufferStudyArea

In [2]:
# Create a GIS Connection
gis = GIS(profile='your_online_profile')

In [3]:
# Get US as a country
usa = Country.get('US')
type(usa)

arcgis.geoenrichment.enrichment.Country

In [4]:
df = usa.data_collections

# print a few rows of the DataFrame
df.head()

,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
1yearincrements,1yearincrements.AGE0_CY,2022 Population Age <1,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE1_CY,2022 Population Age 1,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE2_CY,2022 Population Age 2,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE3_CY,2022 Population Age 3,2022 Age: 1 Year Increments (Esri),2022
1yearincrements,1yearincrements.AGE4_CY,2022 Population Age 4,2022 Age: 1 Year Increments (Esri),2022


In [5]:
# call the shape property to get the total number of rows and columns
df.shape

(18946, 4)

Each data collection can have multiple analysis variables as seen in the table above. Every such analysis variable has a unique ID, found in the `analysisVariable` column. When calling the `enrich()` method, these analysis variables can be passed in the `data_collections` and `analysis_variables` parameters.

You can filter the `data_collections` and query the collections `analysis_variables` using Pandas expressions.

In [6]:
# get all the unique data collections available for the current country
df.index.unique()

Index(['1yearincrements', '5yearincrements', 'Age', 'agebyracebysex',
       'AgeDependency', 'AtRisk', 'AutomobilesAutomotiveProducts',
       'BabyProductsToysGames', 'basicFactsForMobileApps', 'businesses',
       ...
       'travelMPI', 'unitsinstructure', 'urbanizationgroupsNEW', 'vacant',
       'vehiclesavailable', 'veterans', 'Wealth', 'women', 'yearbuilt',
       'yearmovedin'],
      dtype='object', name='dataCollectionID', length=115)

The snippet below shows how you can query the `Age` data collection and get all the unique `analysisVariable`s under that collection.

In [7]:
df.loc['Age']['analysisVariable'].unique()

array(['Age.MALE0', 'Age.MALE5', 'Age.MALE10', 'Age.MALE15', 'Age.MALE20',
       'Age.MALE25', 'Age.MALE30', 'Age.MALE35', 'Age.MALE40',
       'Age.MALE45', 'Age.MALE50', 'Age.MALE55', 'Age.MALE60',
       'Age.MALE65', 'Age.MALE70', 'Age.MALE75', 'Age.MALE80',
       'Age.MALE85', 'Age.FEM0', 'Age.FEM5', 'Age.FEM10', 'Age.FEM15',
       'Age.FEM20', 'Age.FEM25', 'Age.FEM30', 'Age.FEM35', 'Age.FEM40',
       'Age.FEM45', 'Age.FEM50', 'Age.FEM55', 'Age.FEM60', 'Age.FEM65',
       'Age.FEM70', 'Age.FEM75', 'Age.FEM80', 'Age.FEM85'], dtype=object)

In [8]:
# View a sample of the `Age` data collection
df.loc['Age'].head()

,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
Age,Age.MALE0,2022 Males Age 0-4,2022 Age: 5 Year Increments (Esri),2022
Age,Age.MALE5,2022 Males Age 5-9,2022 Age: 5 Year Increments (Esri),2022
Age,Age.MALE10,2022 Males Age 10-14,2022 Age: 5 Year Increments (Esri),2022
Age,Age.MALE15,2022 Males Age 15-19,2022 Age: 5 Year Increments (Esri),2022
Age,Age.MALE20,2022 Males Age 20-24,2022 Age: 5 Year Increments (Esri),2022


Now, let's look at some examples of enriching each of the study areas.

### Enriching street address
Street address locations can be passed as strings of input street addresses, points of interest or place names. A street address can be passed as a single line or as a multiple field input. If a point (e.g. a street address) is used as a study area, the service will create a 1 mile ring buffer around the point to collect and append enrichment data. 

The example below uses a `street address` as a study area for enrichment using `Age` data collection.

#### Single line address

In [9]:
# Enriching single address as single line imput
single_address = enrich(study_areas=["380 New York St Redlands CA 92373"], 
                       data_collections=['Age'])

In [10]:
single_address

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,-117.19479,34.057265,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,366.0,392.0,365.0,345.0,322.0,277.0,168.0,103.0,132.0,"{""rings"": [[[-117.19479001927878, 34.071773611..."


##### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [11]:
# Plot on a map
address_map = gis.map('Redlands, CA',13)
address_map

MapView(layout=Layout(height='400px', width='100%'))

A buffer of 1 mile is created by default, as seen on this map, for any address.

In [12]:
single_address.spatial.plot(address_map)

True

__Multiple addresses as single line input__

In [13]:
# Enriching multiple addresses as single line input
enrich(study_areas=[{"address":{"text":"12 Concorde Place Toronto ON M3C 3R8","sourceCountry":"Canada"}},
                    {"address":{"text":"380 New York St Redlands CA 92373","sourceCountry":"USA"}}], 
       data_collections=['Age'])

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,...,ecypfa4549,ecypfa5054,ecypfa5559,ecypfa6064,ecypfa6569,ecypfa7074,ecypfa7579,ecypfa8084,ecypfa85_p,SHAPE
0,CAN,-79.328779,43.729724,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:CAN.DA;PointsLayer:CAN.Bloc...,-1.0,-1.0,...,1307.0,1256.0,1282.0,1165.0,1175.0,1035.0,857.0,597.0,978.0,"{""rings"": [[[-79.32877900047636, 43.7442083312..."
1,CAN,-117.19479,34.057265,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:CAN.DA;PointsLayer:CAN.Bloc...,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{""rings"": [[[-117.19479001927878, 34.071773611..."


#### Multiple field input

In [14]:
enrich(study_areas=[{"address":{"Address":"380 New York Street", 
                                "City":"Redlands", "Region":"CA", "Postal":92373}}], 
       data_collections=['Age'])

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,-117.19479,34.057265,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,366.0,392.0,365.0,345.0,322.0,277.0,168.0,103.0,132.0,"{""rings"": [[[-117.19479001927878, 34.071773611..."


Enriching with various analysis variables for age such as `FEM45, FEM50, FEM65` etc.

In [15]:
enrich(study_areas=["380 New York St Redlands CA 92373"], 
       analysis_variables=["Age.FEM45","Age.FEM55","Age.FEM65"])

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,fem45,fem55,fem65,SHAPE
0,USA,-117.19479,34.057265,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,366.0,365.0,322.0,"{""rings"": [[[-117.19479001927878, 34.071773611..."


### Enriching point, line and polygon geometries

Point geometries can be passed as x and y coordinates to `study_areas` parameter. When points are specified as study areas, the service will analyze map areas surrounding or associated with the input point locations. Unless otherwise specified, the service will analyze a one mile ring around a point. This is also true for a line. Locations can also be given as polygon geometries.

#### Single Point described as map coordinates

In [16]:
from arcgis.geometry import Point

In [17]:
pt = Point({"x" : -117.1956, "y" : 34.0572, "spatialReference" : {"wkid" : 4326}})
enrich(study_areas=[pt], data_collections=['Age'])

,source_country,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,462.0,...,352.0,378.0,350.0,333.0,310.0,268.0,160.0,98.0,125.0,"{""rings"": [[[-117.19559999999998, 34.071708616..."


#### Multiple points with attributes described as map coordinates

In [18]:
pt1 = Point({"x" : -122.435, "y" : 37.785, "spatialReference" : {"wkid" : 4326}})
pt2 = Point({"x" : -122.433, "y" : 37.734, "spatialReference" : {"wkid" : 4326}})

enrich(study_areas=[pt1, pt2], data_collections=['Age'])

,source_country,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1670.0,...,3013.0,2560.0,2535.0,2814.0,2677.0,2562.0,2096.0,1650.0,2327.0,"{""rings"": [[[-122.43499999999999, 37.799499596..."
1,USA,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,1554.0,...,2340.0,2273.0,2239.0,2134.0,2034.0,1726.0,1183.0,821.0,864.0,"{""rings"": [[[-122.43299999999999, 37.748499722..."


#### Line feature described as geometry

In [19]:
from arcgis.geometry import Polyline

In [20]:
line = Polyline({"paths":[[[-13048580,4036370],[-13046151,4036366]]],
                 "spatialReference":{"wkid":102100}})
enriched_line_df = enrich(study_areas=[line], data_collections=['Age'])

In [21]:
enriched_line_df

,source_country,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,RingBuffer,esriMiles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,646.0,...,554.0,553.0,506.0,496.0,450.0,398.0,247.0,163.0,258.0,"{""rings"": [[[-117.21736177272676, 34.070851408..."


##### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [22]:
# Plot on a map
line_map = gis.map('Redlands, CA',13)
line_map

MapView(layout=Layout(height='400px', width='100%'))

We can clearly see the line and a 1 mile buffer around the line in this map

In [23]:
# Draw line
line_map.draw(line)

# Plot enriched area around line
enriched_line_df.spatial.plot(line_map)

True

#### Map area described as polygons

In [24]:
from arcgis.geometry import Polygon

In [25]:
poly = Polygon({"rings":[[[-117.185412,34.063170],[-122.81,37.81],
                        [-117.200570,34.057196],[-117.185412,34.063170]]],
                        "spatialReference":{"wkid":4326}})

enrich(study_areas=[poly], data_collections=['Age'])

,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,male5,male10,male15,male20,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,5430.0,5408.0,5184.0,5105.0,5001.0,...,3724.0,3770.0,3727.0,3517.0,2780.0,2157.0,1447.0,911.0,1013.0,"{""rings"": [[[-117.18541199999999, 34.063170000..."


### Enriching Buffered study areas

`BufferStudyArea` instances are used to change the ring buffer size or create drive-time service areas around points specified using one of the above methods. BufferStudyArea allows you to buffer point and street address study areas. They can be created using the following parameters:
        * area: the point geometry or street address (string) study area to be buffered
        * radii: list of distances by which to buffer the study area, eg. [1, 2, 3]
        * units: distance unit, eg. Miles, Kilometers, Minutes (when using drive times/travel_mode)
        * overlap: boolean, uses overlapping rings/network service areas when True, or non-overlapping disks when False
        * travel_mode: None or string, one of the supported travel modes when using network service areas
        

`BufferStudyArea` also allows you to define drive time service areas around points as well as other advanced service areas such as walking and trucking.

#### Buffering location using driving distance 

The example below creates disks of radii 1, 3 and 5 Miles respectively from a street address and enriches these using the 'Age' data collection.

In [26]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373',
                           radii=[1,3,5], units='Miles', overlap=False)
drive_dist_df = enrich(study_areas=[buffered], data_collections=['Age'])

In [27]:
drive_dist_df

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,-117.19479,34.057265,RingBuffer,Miles,Miles,1.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,366.0,392.0,365.0,345.0,322.0,277.0,168.0,103.0,132.0,"{""rings"": [[[-117.19479001927878, 34.071773611..."
1,USA,-117.19479,34.057265,RingBuffer,Miles,Miles,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,2336.0,2361.0,2423.0,2514.0,2218.0,1811.0,1283.0,899.0,1184.0,"{""rings"": [[[-117.19479001927878, 34.100790740..."
2,USA,-117.19479,34.057265,RingBuffer,Miles,Miles,5.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,4717.0,4841.0,5002.0,5009.0,4336.0,3444.0,2473.0,1723.0,2258.0,"{""rings"": [[[-117.19479001927878, 34.129807732..."


##### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [28]:
# Plot on a map
buffer_map1 = gis.map('Redlands, CA')
buffer_map1.basemap = 'dark-gray-vector'
buffer_map1

MapView(layout=Layout(height='400px', width='100%'))

In [29]:
drive_dist_df.spatial.plot(map_widget=buffer_map1,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=4,  # choose the number of classes
               col='bufferRadii',  # numeric column to classify
               cmap='viridis',  # color map to pick colors from for each class
               alpha=0.7  # specify opacity
               )

True

#### Buffering location using drive times

The example below creates 5 and 10 minute drive times from a street address and enriches these using the 'Age' data collection.

In [30]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373', 
                           radii=[5, 10], units='Minutes', 
                           travel_mode='Driving')
drive_time_df = enrich(study_areas=[buffered], data_collections=['Age'])

In [31]:
drive_time_df

,source_country,x,y,area_type,buffer_units,buffer_units_alias,buffer_radii,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,USA,-117.19479,34.057265,NetworkServiceArea,Minutes,Drive Time Minutes,5.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,457.0,479.0,461.0,434.0,404.0,342.0,215.0,137.0,175.0,"{""rings"": [[[-117.19996384486102, 34.076732195..."
1,USA,-117.19479,34.057265,NetworkServiceArea,Minutes,Drive Time Minutes,10.0,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,...,2831.0,2860.0,2954.0,3025.0,2642.0,2155.0,1537.0,1111.0,1604.0,"{""rings"": [[[-117.19300193285012, 34.134897844..."


##### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [32]:
# Plot on a map
buffer_map2 = gis.map('Redlands, CA')
buffer_map2.basemap = 'dark-gray-vector'
buffer_map2

MapView(layout=Layout(height='400px', width='100%'))

In [33]:
drive_time_df.spatial.plot(map_widget=buffer_map2,
                   renderer_type='c',  # for class breaks renderer
                   method='esriClassifyNaturalBreaks',  # classification algorithm
                   class_count=3,  # choose the number of classes
                   col='bufferRadii',  # numeric column to classify
                   cmap='viridis',  # color map to pick colors from for each class
                   alpha=0.7  # specify opacity
                   )

True

### Enriching a named statistical area

In all previous examples of different study area types, locations were defined as either points or polygons. Study area locations can also be passed as one or many named statistical areas. This form of study area lets you define an area as a standard geographic statistical feature, such as a census or postal area, for example, to obtain enrichment information for a U.S. state, county, or ZIP Code or a Canadian province or postal code. We will explore Named statistical areas in detail in the next section.


#### Enriching a zip code
Enriching zip code 92373 in California using the 'Age' data collection:

In [34]:
usa = Country.get('US')

In [35]:
redlands = usa.subgeographies.states['California'].zip5['92373']

In [36]:
type(redlands)

arcgis.geoenrichment.enrichment.NamedArea

In [37]:
redlands

<NamedArea name:"United States" area_id="92373", level="US.ZIP5", country="United States">

In [38]:
redlands_df = enrich(study_areas=[redlands], data_collections=['Age'] )

In [39]:
redlands_df

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,male5,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,US.ZIP5,Redlands,92373,USA,Query:US.ZIP5,2.191,2.576,1,888.0,911.0,...,1056.0,1135.0,1152.0,1209.0,1149.0,1014.0,730.0,518.0,810.0,"{""rings"": [[[-117.16767396036383, 33.976847519..."



##### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [40]:
zip_map = gis.map('Redlands, CA')
zip_map

MapView(layout=Layout(height='400px', width='100%'))

In [41]:
redlands_df.spatial.plot(zip_map)

True

#### Enriching all counties in a state

##### Note: When getting NamedAreas in general a dictionary is returned. In this case, you can either pass in the dictionary as a variable to the study_areas parameter or a list of the dictionary values.   
Here is an example of each case:

In [42]:
ca_counties = usa.subgeographies.states['California'].counties

In [43]:
counties_df = enrich(study_areas=ca_counties, data_collections=['Age'])
counties_df.head()

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,male0,male5,...,fem45,fem50,fem55,fem60,fem65,fem70,fem75,fem80,fem85,SHAPE
0,US.Counties,Alameda County,06001,USA,Query:US.Counties,2.191,2.576,1,49161.0,51493.0,...,55521.0,54569.0,55358.0,53052.0,46019.0,37534.0,25991.0,16889.0,19975.0,"{""rings"": [[[-122.27036220796694, 37.904395965..."
1,US.Counties,Alpine County,06003,USA,Query:US.Counties,2.191,2.576,1,33.0,37.0,...,31.0,42.0,50.0,44.0,68.0,30.0,23.0,15.0,8.0,"{""rings"": [[[-119.90060896152161, 38.930765231..."
2,US.Counties,Amador County,06005,USA,Query:US.Counties,2.191,2.576,1,685.0,787.0,...,1040.0,1181.0,1553.0,1851.0,1739.0,1579.0,1040.0,697.0,690.0,"{""rings"": [[[-120.07765196234347, 38.708892198..."
3,US.Counties,Butte County,06007,USA,Query:US.Counties,2.191,2.576,1,5738.0,5744.0,...,5335.0,5784.0,6706.0,7248.0,6928.0,5700.0,4003.0,2574.0,3220.0,"{""rings"": [[[-121.40463436180775, 40.146646214..."
4,US.Counties,Calaveras County,06009,USA,Query:US.Counties,2.191,2.576,1,870.0,942.0,...,1205.0,1510.0,1968.0,2198.0,2257.0,1876.0,1172.0,657.0,617.0,"{""rings"": [[[-120.07247293757524, 38.509161181..."


In [44]:
counties_df2 = usa.enrich(study_areas=list(ca_counties.values()), data_collections=['transportation'])
counties_df2.head()

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,x6001_x,x6001_a,...,x6061fy_x,x6061fy_a,x6061fy_i,x6062fy_x,x6062fy_a,x6062fy_i,x6063fy_x,x6063fy_a,x6063fy_i,SHAPE
0,US.Counties,Alameda County,06001,USA,Query:US.Counties,2.191,2.576,1,8938340557.0,14920.26,...,192310449.0,325.29,242.0,115477218.0,195.33,202.0,2607315.0,4.41,181.0,"{""rings"": [[[-122.27036220796694, 37.904395965..."
1,US.Counties,Alpine County,06003,USA,Query:US.Counties,2.191,2.576,1,5114164.0,9759.85,...,41728.0,79.33,59.0,36960.0,70.27,73.0,775.0,1.47,60.0,"{""rings"": [[[-119.90060896152161, 38.930765231..."
2,US.Counties,Amador County,06005,USA,Query:US.Counties,2.191,2.576,1,151573467.0,9599.33,...,1328685.0,82.75,62.0,1115814.0,69.49,72.0,22837.0,1.42,58.0,"{""rings"": [[[-120.07765196234347, 38.708892198..."
3,US.Counties,Butte County,06007,USA,Query:US.Counties,2.191,2.576,1,837128815.0,9924.94,...,9600470.0,113.27,84.0,8498160.0,100.27,104.0,174002.0,2.05,84.0,"{""rings"": [[[-121.40463436180775, 40.146646214..."
4,US.Counties,Calaveras County,06009,USA,Query:US.Counties,2.191,2.576,1,182566082.0,9829.12,...,1417301.0,76.45,57.0,1300752.0,70.16,72.0,26749.0,1.44,59.0,"{""rings"": [[[-120.07247293757524, 38.509161181..."


##### Visualize results on a map

In [45]:
county_map = gis.map('California')
county_map

MapView(layout=Layout(height='400px', width='100%'))

In [46]:
counties_df.spatial.plot(map_widget=county_map,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=5,  # choose the number of classes
               col='FEM75',  # numeric column to classify
               cmap='viridis',  # color map to pick colors from for each class
               alpha=0.7  # specify opacity
               )

True

In [47]:
county_map.legend=True

## Conclusion

In this part of the `arcgis.geoenrichment` module guide series, you were introduced to the concept of study areas and how Geoenrichment uses a study area to define the location of the point, polyline or area that you want to enrich. You have also seen in detail how different types of study areas can be enriched and visualized on a map.

In the subsequent pages, you will learn about:
1. Exploring Named Statistical Areas (explains where to enrich continued)
2. Data Collections and GeoEnrichment coverage (explains what datasets/variables to enrich with)
3. Generating Reports
4. Standard Geography Queries